In [1]:
import numpy as np
import pandas as pd

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ods-churn-24/currency_rk.csv
/kaggle/input/ods-churn-24/report_dates.csv
/kaggle/input/ods-churn-24/mcc_codes.csv
/kaggle/input/ods-churn-24/clients.csv
/kaggle/input/ods-churn-24/train.csv
/kaggle/input/ods-churn-24/sample_submit_naive.csv
/kaggle/input/ods-churn-24/transactions.csv


In [2]:
import pandas as pd
import numpy as np
# from sksurv.ensemble import RandomSurvivalForest
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import random
import warnings
from IPython.display import display, HTML

# pd.set_option('display.max_columns', None)
warnings.simplefilter('ignore')
# pd.options.display.max_columns = 100
# pd.options.display.max_rows = 100
random.seed(42)
np.random.seed(42)

In [3]:
clients = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
report_dates = pd.read_csv('/kaggle/input/ods-churn-24/report_dates.csv', parse_dates=['report_dt'])
transactions = pd.read_csv('/kaggle/input/ods-churn-24/transactions.csv', parse_dates=['transaction_dttm'])
currency_rk = pd.read_csv('/kaggle/input/ods-churn-24/currency_rk.csv')
currency_mult = pd.DataFrame({'currency_rk': [0,1,2,3], 'mult':[24,1,100,90]})
mcc_codes = pd.read_csv('/kaggle/input/ods-churn-24/mcc_codes.csv')
sample_submit_naive = pd.read_csv('/kaggle/input/ods-churn-24/sample_submit_naive.csv')
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')

transactions = transactions.merge(currency_mult, how='left')
transactions.transaction_amt = transactions.transaction_amt * transactions.mult
transactions.drop(columns=['mult'], inplace=True)

#### FeatureTools

In [4]:
import featuretools as ft

# Load the datasets
train_df = clients[['user_id']].copy()
transactions_df = transactions.drop(['currency_rk'], axis=1).copy()

# train_df = clients[['user_id']][0:500].copy()
# transactions_df = transactions[0:5000].drop(['currency_rk'], axis=1).copy()

# Convert transaction_dttm to datetime
transactions_df['transaction_dttm'] = pd.to_datetime(transactions_df['transaction_dttm'])

# Create an EntitySet
es = ft.EntitySet(id='user_transactions')

# Add the transactions dataframe to the EntitySet
es = es.add_dataframe(
    dataframe_name='transactions',
    dataframe=transactions_df,
    index='transaction_id',
    time_index='transaction_dttm',
    logical_types={
        'mcc_code': 'Categorical',
        'transaction_amt': 'Double',  # Corrected logical type
        'transaction_dttm': 'Datetime'
    }
)

# Add the train dataframe to the EntitySet
es = es.add_dataframe(
    dataframe_name='train',
    dataframe=train_df,
    index='user_id'
)

# Correctly defining the relationship
es = es.add_relationships([("train", "user_id", "transactions", "user_id")])

# Set interesting values (example)
# transactions_df['mcc_code'].value_counts().nlargest(10).index.tolist() could be used to find interesting mcc_codes
# Here, you manually define them or use a method to find them dynamically
interesting_mcc_codes = transactions['mcc_code'].value_counts(normalize=True).index.tolist()  # Example MCC codes, replace with relevant codes for your case
es['transactions']['mcc_code'].interesting_values = interesting_mcc_codes

# Automatically generate features with expanded primitives and max_depth
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name='train',
    agg_primitives=[
        'sum', 'mean', 'max', 'min', 'count', 'percent_true', 'num_unique', 'mode', 'skew',
        'trend', 'std'
    ],  # Expanded list of aggregation primitives
    trans_primitives=[
        'day', 'month', 'year', 'weekday', 'is_weekend',
        'time_since_previous', 'cum_sum', 'percentile', 'diff'
    ],  # Expanded list of transformation primitives
    where_primitives=['sum', 'mean', 'max'],  # Conditional aggregations
    max_depth=3,  # Increased max_depth for more complex features
)

In [5]:
main_ft = clients[['user_id']].copy()
main_ft = main_ft.merge(feature_matrix, how='left', on='user_id')
main_ft

,user_id,COUNT(transactions),MAX(transactions.transaction_amt),MEAN(transactions.transaction_amt),MIN(transactions.transaction_amt),MODE(transactions.mcc_code),NUM_UNIQUE(transactions.mcc_code),SKEW(transactions.transaction_amt),STD(transactions.transaction_amt),SUM(transactions.transaction_amt),...,PERCENTILE(STD(transactions.PERCENTILE(transaction_amt))),PERCENTILE(STD(transactions.TIME_SINCE_PREVIOUS(transaction_dttm))),PERCENTILE(SUM(transactions.CUM_SUM(transaction_amt))),PERCENTILE(SUM(transactions.DIFF(transaction_amt))),PERCENTILE(SUM(transactions.PERCENTILE(transaction_amt))),PERCENTILE(SUM(transactions.TIME_SINCE_PREVIOUS(transaction_dttm))),"PERCENTILE(TREND(transactions.CUM_SUM(transaction_amt), transaction_dttm))","PERCENTILE(TREND(transactions.DIFF(transaction_amt), transaction_dttm))","PERCENTILE(TREND(transactions.PERCENTILE(transaction_amt), transaction_dttm))","PERCENTILE(TREND(transactions.TIME_SINCE_PREVIOUS(transaction_dttm), transaction_dttm))"
0,3,11,104011.960938,1246.037876,-153866.890625,0,4,-1.399640,60860.880477,13706.416641,...,0.959031,0.075479,0.988750,0.530344,0.091469,0.033969,0.770104,0.969542,0.959490,0.351208
1,9,90,-45.579891,-3593.718520,-90147.617188,1,22,-5.877426,11797.055119,-323434.666813,...,0.145302,0.833292,0.811604,0.187344,0.394917,0.721422,0.878115,0.210271,0.200677,0.073656
2,13,22,258547.195312,5593.500744,-58740.300781,3,4,3.689363,61083.237782,123057.016357,...,0.982323,0.857802,0.855229,0.666562,0.104073,0.281443,0.421740,0.063583,0.423094,0.143000
3,37,315,5487.140625,-1053.522538,-35782.984375,2,28,-6.201203,3913.114835,-331859.599463,...,0.289760,0.627594,0.163344,0.128177,0.907896,0.909474,0.504417,0.585948,0.592177,0.197500
4,41,16,-290.766998,-6786.663385,-16841.208984,3,5,-0.291520,5251.737982,-108586.614166,...,0.056208,0.814073,0.991042,0.252823,0.021792,0.202208,0.693719,0.869333,0.750052,0.038458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,37,-40.769005,-799.493409,-13020.519531,1,12,-5.497509,2136.038913,-29581.256115,...,0.470698,0.989729,0.535417,0.460594,0.312208,0.427542,0.886938,0.811625,0.948604,0.980010
95996,562205,151,1595.461060,-268.158335,-6220.171387,11,15,-6.944679,590.682233,-40491.908630,...,0.331094,0.812937,0.147448,0.292406,0.783885,0.792807,0.956000,0.481000,0.294208,0.903667
95997,562312,56,-28.292030,-331.032523,-1372.377075,1,7,-1.880765,309.587795,-18537.821270,...,0.125208,0.862521,0.407865,0.272000,0.448583,0.541943,0.922531,0.169583,0.584313,0.879896
95998,562721,85,5412.773926,-1929.467785,-18981.269531,3,11,-1.777065,4172.623511,-164004.761685,...,0.915031,0.969094,0.291906,0.284615,0.521635,0.871708,0.922698,0.258698,0.043240,0.853625


#### TSFresh

In [6]:
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.utilities.dataframe_functions import impute

# Load the transaction data
transactions_df = transactions.drop(['currency_rk'], axis=1).copy()

# transactions_df = transactions[0:5000].drop(['currency_rk'], axis=1).copy()

# Ensure 'transaction_dttm' is a datetime type, assuming it's already in an appropriate datetime format
transactions_df['transaction_dttm'] = pd.to_datetime(transactions_df['transaction_dttm'])

# Setting up the data in a format suitable for TSFresh
# Assuming 'user_id' is in your transactions_df to identify different users
# We will use 'transaction_dttm' as the time index
transactions_df_sorted = transactions_df.sort_values(by=['user_id', 'transaction_dttm'])

# Extract features
extraction_settings = ComprehensiveFCParameters()

# The 'column_id' is assumed to be 'user_id' to treat each user's data as a separate time series
# The 'column_sort' is 'transaction_dttm' to sort transactions in time order
# 'column_value' could be 'transaction_amt' if you're interested in extracting features from the transaction amounts
extracted_features = extract_features(
    transactions_df_sorted,
    column_id='user_id',
    column_sort='transaction_dttm',
    column_value='transaction_amt',  # Or any other column you want to analyze
    default_fc_parameters=extraction_settings,
    impute_function=impute  # Impute missing values generated during feature extraction
)

Feature Extraction: 100%|██████████| 10/10 [2:50:55<00:00, 1025.58s/it]


In [7]:
extracted_features['user_id'] = extracted_features.index

main_tsf = clients[['user_id']].copy()
main_tsf = main_tsf.merge(extracted_features, how='left', on='user_id')
main_tsf

,user_id,transaction_amt__variance_larger_than_standard_deviation,transaction_amt__has_duplicate_max,transaction_amt__has_duplicate_min,transaction_amt__has_duplicate,transaction_amt__sum_values,transaction_amt__abs_energy,transaction_amt__mean_abs_change,transaction_amt__mean_change,transaction_amt__mean_second_derivative_central,...,transaction_amt__fourier_entropy__bins_5,transaction_amt__fourier_entropy__bins_10,transaction_amt__fourier_entropy__bins_100,transaction_amt__permutation_entropy__dimension_3__tau_1,transaction_amt__permutation_entropy__dimension_4__tau_1,transaction_amt__permutation_entropy__dimension_5__tau_1,transaction_amt__permutation_entropy__dimension_6__tau_1,transaction_amt__permutation_entropy__dimension_7__tau_1,transaction_amt__query_similarity_count__query_None__threshold_0.0,transaction_amt__mean_n_absolute_max__number_of_maxima_7
0,3,1.0,0.0,0.0,0.0,13706.416641,3.705755e+10,48629.881075,10419.584489,5647.084165,...,0.867563,1.329661,1.791759,1.581094,1.732868,1.747868,1.791759,1.609438,0.0,50005.129116
1,9,1.0,0.0,0.0,0.0,-323434.666813,1.354851e+10,5642.431551,-37.723483,-8.910561,...,1.217997,1.766259,3.257936,1.770063,2.969948,4.004814,4.377414,4.430817,0.0,31480.165667
2,13,1.0,0.0,0.0,0.0,123057.016357,7.904272e+10,49278.751256,134.785063,-7624.287256,...,1.424130,1.820076,2.369382,1.712732,2.305657,2.736339,2.751667,2.772589,0.0,67676.247210
3,37,1.0,0.0,0.0,0.0,-331859.599463,5.157736e+09,1758.458152,-5.398915,-3.303684,...,1.411973,2.089130,3.993870,1.790625,3.155078,4.561627,5.405563,5.654786,0.0,25445.313058
4,41,1.0,0.0,0.0,0.0,-108586.614166,1.150652e+09,5437.855245,707.660254,-346.270630,...,0.936888,1.522955,2.197225,1.673118,2.351673,2.484907,2.397895,2.302585,0.0,11681.743513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,1.0,0.0,0.0,0.0,-29581.256115,1.879059e+08,1155.361053,48.327661,158.056179,...,0.826405,1.128596,2.406160,1.765433,2.771655,3.328472,3.465736,3.433987,0.0,3139.546047
95996,562205,1.0,0.0,0.0,0.0,-40491.908630,6.319407e+07,397.368981,-3.170353,0.318767,...,1.016382,1.592042,3.491539,1.772070,3.054030,4.284325,4.805526,4.938491,0.0,1994.845040
95997,562312,1.0,0.0,0.0,0.0,-18537.821270,1.140807e+07,276.899329,2.602364,-18.640738,...,1.382994,1.974293,3.176083,1.726918,2.901163,3.594608,3.823097,3.884297,0.0,1021.007141
95998,562721,1.0,0.0,0.0,0.0,-164004.761685,1.778948e+09,4398.608403,-38.972757,83.021431,...,1.411471,2.063792,3.503285,1.717146,2.976880,3.968511,4.382027,4.369448,0.0,12292.178850


#### Selection

In [8]:
data = main_ft.merge(train[['user_id', 'target']], how='left')
data = data[~data.target.isna()]
labels = data['target']
data = data.drop(columns=['target', 'user_id'])

print(main_ft.shape)

(96000, 174)


In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold

# Load the datasets
features_df = data.copy()
target_df = labels.copy()

# Assuming 'user_id' is the common key and both datasets are aligned
# If not, you might need to merge or align them based on your specific needs

# Step 1: Remove features with a big share of Missing Values
# Let's assume "a big share" means more than 60% missing
threshold = 0.6  # 60%
features_df = features_df.loc[:, features_df.isnull().mean() < threshold]

# Step 2: Remove features with Single Unique Value
features_df = features_df.loc[:, features_df.apply(pd.Series.nunique) != 1]

# Step 3: Remove duplicate features
# Transpose the dataframe, drop duplicate rows (now columns), and transpose back
features_df = features_df.T.drop_duplicates().T

# Step 4: Remove collinear features
# Calculate the correlation matrix and remove one of two features with correlation higher than a threshold
corr_matrix = features_df.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
features_df = features_df.drop(columns=to_drop)

# If necessary, align the target with the features based on 'user_id' or another identifier
# This step is skipped here but can be performed as needed

# Step 5: Optionally, remove features with low variance
# This requires fitting the data, so ensure to handle missing values if any remain
# Fill missing values with 0, mean, median, or any other method as appropriate for your dataset
features_df.fillna(0, inplace=True)  # Example: filling with 0
selector = VarianceThreshold(threshold=0.01)  # Adjust threshold as needed
features_reduced = selector.fit_transform(features_df)

# Convert back to DataFrame if necessary, for further use
features_final = pd.DataFrame(features_reduced, columns=features_df.columns[selector.get_support()])
good_cols = features_final.columns

main_ft = main_ft[['user_id']+list(good_cols)]
print(main_ft.shape)

(96000, 113)


In [10]:
data = main_tsf.merge(train[['user_id', 'target']], how='left')
data = data[~data.target.isna()]
labels = data['target']
data = data.drop(columns=['target', 'user_id'])

print(main_tsf.shape)

(96000, 784)


In [11]:
# Load the datasets
features_df = data.copy()
target_df = labels.copy()

# Assuming 'user_id' is the common key and both datasets are aligned
# If not, you might need to merge or align them based on your specific needs

# Step 1: Remove features with a big share of Missing Values
# Let's assume "a big share" means more than 60% missing
threshold = 0.6  # 60%
features_df = features_df.loc[:, features_df.isnull().mean() < threshold]

# Step 2: Remove features with Single Unique Value
features_df = features_df.loc[:, features_df.apply(pd.Series.nunique) != 1]

# Step 3: Remove duplicate features
# Transpose the dataframe, drop duplicate rows (now columns), and transpose back
features_df = features_df.T.drop_duplicates().T

# Step 4: Remove collinear features
# Calculate the correlation matrix and remove one of two features with correlation higher than a threshold
corr_matrix = features_df.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
features_df = features_df.drop(columns=to_drop)

# If necessary, align the target with the features based on 'user_id' or another identifier
# This step is skipped here but can be performed as needed

# Step 5: Optionally, remove features with low variance
# This requires fitting the data, so ensure to handle missing values if any remain
# Fill missing values with 0, mean, median, or any other method as appropriate for your dataset
features_df.fillna(0, inplace=True)  # Example: filling with 0
selector = VarianceThreshold(threshold=0.01)  # Adjust threshold as needed
features_reduced = selector.fit_transform(features_df)

# Convert back to DataFrame if necessary, for further use
features_final = pd.DataFrame(features_reduced, columns=features_df.columns[selector.get_support()])
good_cols = features_final.columns

main_tsf = main_tsf[['user_id']+list(good_cols)]
print(main_tsf.shape)

(96000, 658)


#### Importance

In [12]:
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
train = main_ft.merge(train[['user_id', 'target']], how='left')
train = train[~train.target.isna()]
train = train.sort_values('user_id').reset_index(drop=True)

cat_cols = main_ft.select_dtypes(exclude=["number","bool_","object_"]).columns.to_list()

# Обучение модельки для того чтобы получить важные фичи

model = CatBoostClassifier(
    iterations = 1400,
    depth=5,
    learning_rate=0.03,
    
    eval_metric='AUC',
    cat_features = cat_cols,
    thread_count=6,
    early_stopping_rounds=200,
)
model.fit(train.drop(['user_id', 'target'], axis=1), train['target'], verbose=100)


df_imp = pd.DataFrame({
    'name': train.drop(['user_id', 'target'], axis=1).columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

df_imp = df_imp[df_imp['imp'] > 0.15] # Берем все фичи, у которых важность больше 0.3
df_imp_050 = df_imp[df_imp['imp'] > 0.50]
df_imp_055 = df_imp[df_imp['imp'] > 0.55]
df_imp_060 = df_imp[df_imp['imp'] > 0.60]
df_imp_065 = df_imp[df_imp['imp'] > 0.65]

good_cols = df_imp['name'].tolist()
good_cols_050 = df_imp_050['name'].tolist()
good_cols_055 = df_imp_055['name'].tolist()
good_cols_060 = df_imp_060['name'].tolist()
good_cols_065 = df_imp_065['name'].tolist()

pred = model.predict_proba(train.drop(['user_id', 'target'], axis=1))[:, 1]
print(metrics.roc_auc_score(train['target'], pred))

TBB Warning: The number of workers is currently limited to 3. The request for 5 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



0:	total: 153ms	remaining: 3m 34s
100:	total: 9.53s	remaining: 2m 2s
200:	total: 18.3s	remaining: 1m 48s
300:	total: 27s	remaining: 1m 38s
400:	total: 35.8s	remaining: 1m 29s
500:	total: 44.1s	remaining: 1m 19s
600:	total: 52.6s	remaining: 1m 9s
700:	total: 1m 1s	remaining: 1m 1s
800:	total: 1m 10s	remaining: 52.8s
900:	total: 1m 19s	remaining: 43.8s
1000:	total: 1m 27s	remaining: 34.9s
1100:	total: 1m 35s	remaining: 26s
1200:	total: 1m 44s	remaining: 17.3s
1300:	total: 1m 53s	remaining: 8.6s
1399:	total: 2m 2s	remaining: 0us
0.8336344108178518


In [13]:
df_imp[0:50]

,name,imp
41,CUM_SUM(COUNT(transactions)),5.713370
33,STD(transactions.CUM_SUM(transaction_amt)),4.151950
62,CUM_SUM(MIN(transactions.TIME_SINCE_PREVIOUS(t...,3.941064
61,CUM_SUM(MEAN(transactions.DIFF(transaction_amt))),3.695955
42,"CUM_SUM(TREND(transactions.transaction_amt, tr...",3.433158
10,MAX(transactions.CUM_SUM(transaction_amt)),3.355594
5,NUM_UNIQUE(transactions.mcc_code),3.202233
56,PERCENTILE(NUM_UNIQUE(transactions.mcc_code)),2.278683
25,NUM_UNIQUE(transactions.MONTH(transaction_dttm)),2.210267
110,PERCENTILE(TREND(transactions.PERCENTILE(trans...,1.812180


In [14]:
main_ft = main_ft[['user_id']+good_cols]
main_ft_050 = main_ft[['user_id']+good_cols_050]
main_ft_055 = main_ft[['user_id']+good_cols_055]
main_ft_060 = main_ft[['user_id']+good_cols_060]
main_ft_065 = main_ft[['user_id']+good_cols_065]

In [15]:
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
train = main_tsf.merge(train[['user_id', 'target']], how='left')
train = train[~train.target.isna()]
train = train.sort_values('user_id').reset_index(drop=True)

cat_cols = main_tsf.select_dtypes(exclude=["number","bool_","object_"]).columns.to_list()

# Обучение модельки для того чтобы получить важные фичи

model = CatBoostClassifier(
    iterations = 1400,
    depth=5,
    learning_rate=0.03,
    
    eval_metric='AUC',
#     cat_features = cat_cols,
    thread_count=6,
    early_stopping_rounds=200,
)
model.fit(train.drop(['user_id', 'target'], axis=1), train['target'], verbose=100)


df_imp = pd.DataFrame({
    'name': train.drop(['user_id', 'target'], axis=1).columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

df_imp = df_imp[df_imp['imp'] > 0.15] # Берем все фичи, у которых важность больше 0.3
df_imp_050 = df_imp[df_imp['imp'] > 0.50]
df_imp_055 = df_imp[df_imp['imp'] > 0.55]
df_imp_060 = df_imp[df_imp['imp'] > 0.60]
df_imp_065 = df_imp[df_imp['imp'] > 0.65]

good_cols = df_imp['name'].tolist()
good_cols_050 = df_imp_050['name'].tolist()
good_cols_055 = df_imp_055['name'].tolist()
good_cols_060 = df_imp_060['name'].tolist()
good_cols_065 = df_imp_065['name'].tolist()

pred = model.predict_proba(train.drop(['user_id', 'target'], axis=1))[:, 1]
print(metrics.roc_auc_score(train['target'], pred))

0:	total: 196ms	remaining: 4m 34s
100:	total: 16.3s	remaining: 3m 30s
200:	total: 31.8s	remaining: 3m 9s
300:	total: 47.3s	remaining: 2m 52s
400:	total: 1m 1s	remaining: 2m 33s
500:	total: 1m 16s	remaining: 2m 17s
600:	total: 1m 31s	remaining: 2m 2s
700:	total: 1m 50s	remaining: 1m 50s
800:	total: 2m 4s	remaining: 1m 33s
900:	total: 2m 19s	remaining: 1m 17s
1000:	total: 2m 33s	remaining: 1m 1s
1100:	total: 2m 48s	remaining: 45.8s
1200:	total: 3m 2s	remaining: 30.3s
1300:	total: 3m 17s	remaining: 15s
1399:	total: 3m 32s	remaining: 0us
0.8452948383214207


In [16]:
df_imp[0:50]

,name,imp
22,transaction_amt__maximum,3.806132
571,"transaction_amt__fft_aggregated__aggtype_""kurt...",2.590718
633,transaction_amt__number_crossing_m__m_0,2.335846
369,"transaction_amt__fft_coefficient__attr_""abs""__...",1.784671
643,transaction_amt__energy_ratio_by_chunks__num_s...,1.336497
51,transaction_amt__quantile__q_0.7,1.155182
5,transaction_amt__median,1.085181
53,transaction_amt__quantile__q_0.9,1.009732
159,"transaction_amt__change_quantiles__f_agg_""var""...",0.943667
146,"transaction_amt__change_quantiles__f_agg_""mean...",0.840672


In [17]:
main_tsf = main_tsf[['user_id']+good_cols]
main_tsf_050 = main_tsf[['user_id']+good_cols_050]
main_tsf_055 = main_tsf[['user_id']+good_cols_055]
main_tsf_060 = main_tsf[['user_id']+good_cols_060]
main_tsf_065 = main_tsf[['user_id']+good_cols_065]

In [18]:
main_ft.to_csv('main_ft.csv', index=False)
main_tsf.to_csv('main_tsf.csv', index=False)

In [19]:
main_ft_050.to_csv('main_ft_050.csv', index=False)
main_ft_055.to_csv('main_ft_055.csv', index=False)
main_ft_060.to_csv('main_ft_060.csv', index=False)
main_ft_065.to_csv('main_ft_065.csv', index=False)
main_tsf_050.to_csv('main_tsf_050.csv', index=False)
main_tsf_055.to_csv('main_tsf_055.csv', index=False)
main_tsf_060.to_csv('main_tsf_060.csv', index=False)
main_tsf_065.to_csv('main_tsf_065.csv', index=False)